In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from linearmodels.iv import IV2SLS  
from statsmodels.formula.api import ols
from scipy.optimize import minimize_scalar

# データの準備（第２回と同じコード）

In [2]:
# 自動車データ 
data = pd.read_csv("source/CleanData_20180222.csv", encoding='shift_jis')

# 潜在的な市場規模としての、家計数データ（出所：https://www.airia.or.jp/publish/file/r5c6pv0000006s9v-att/r5c6pv0000006saa.pdf）
dataHH = pd.read_csv("source/HHsize.csv")

# 価格の実質化のための、消費者物価指数（出所：https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031431770&fileKind=1）
dataCPI = pd.read_csv("source/zni2015s.csv", encoding='shift_jis')

# 必要な列のみ抽出・列名の変更
dataCPI = dataCPI[6:56]
dataCPI = dataCPI.rename(columns={'類・品目':'year', '総合':'cpi'})

# 型の変更
dataCPI = dataCPI.astype({'year': 'f2', 'cpi': 'f2'})

# 必要な変数のみ抽出
data = data.loc[:, ['Maker', 'Name', 'Year', 'Sales', 'Model', 'price', 'kata', 'weight', 
                   'FuelEfficiency', 'HorsePower', 'overall_length', 'overall_width','overall_height']]
# 後ほどのマージのために列名の変更
data = data.rename(columns={'Year' : 'year'}) 

# データのマージ
data = pd.merge(data, dataHH,how='left')
data = pd.merge(data, dataCPI,how='left')

# 燃費が欠損しているデータを削除する
#data = data['FuelEfficiency'].dropna()

data = data.dropna()

# 2016年を基準として、価格の実質化を実行する。
# 価格の単位は100万円。

filtered_dataCPI = dataCPI[dataCPI['year'] == 2016]

# 'cpi' 列のみを抽出
cpi2016 = filtered_dataCPI['cpi']

cpi2016 = cpi2016.astype('float64')
data['cpi'] = data['cpi'].astype('float64')
data['price'] = data['price'].astype('float64')

# 'price' 列を 'cpi' 列と 'cpi2016' 列を用いて変換
# data['cpi']/cpi2016が全てNaNとなり、解決できなかったためコメントアウト。 
# 2023/9/16追記：float64に変換することで成功。
# data['price'] = data['price'] / (data['cpi'] / cpi2016) 

# 'price' 列を100で割る
data['price'] = data['price'] / 100

# 'cpi' 列を削除
data = data.drop(columns=['cpi'])

# サイズ(高さ＊幅＊長さ)、燃費の重量に対する比率を定義する。
# 'size' 列を追加
data = data.assign(size=lambda x: (x['overall_length'] / 1000) * (x['overall_width'] / 1000) * (x['overall_height'] / 1000))

# 'hppw' 列を追加
data = data.assign(hppw=lambda x: x['HorsePower'] / x['weight'])

# 'HorsePower' と 'weight' 列を削除
data = data.drop(columns=['HorsePower', 'weight'])

# 列名が 'overall' で始まる列を削除
data = data.drop(columns=data.filter(like='overall').columns)

# 自動車の車種IDを作成する。
grouped_data = data.groupby('Name')
data['NameID'] = grouped_data.ngroup()
data = data.reset_index(drop=True)

# 列の順序を変更 (NameID を year の前に移動)
data = data[['NameID', 'year'] + [col for col in data.columns if col not in ['NameID', 'year']]]

# data['HH']から、コンマを消去、int型に変換する。
data['HH'] = data['HH'].str.replace(',', '').astype(int)

# Market ShareとOutside option shareを定義する。
grouped_data = data.groupby('year')
data['inside_total'] = grouped_data['Sales'].transform('sum')

data = data.assign(outside_total = lambda x: x['HH']-x['inside_total'])
data = data.assign(share = data['Sales']/data['HH'])
data = data.assign(share0 = data['outside_total']/data['HH'])
data = data.drop(columns=['inside_total', 'outside_total'])

# 操作変数の構築

In [3]:
# マーケット・企業レベル（year・Maker）における、各製品属性の和と自乗和を計算する。
data = data.groupby(['year', 'Maker']).apply(lambda x: x.assign(
    hppw_sum_own=x['hppw'].sum(),
    FuelEfficiency_sum_own=x['FuelEfficiency'].sum(),
    size_sum_own=x['size'].sum(),
    hppw_sqr_sum_own=(x['hppw'] ** 2).sum(),
    FuelEfficiency_sqr_sum_own=(x['FuelEfficiency'] ** 2).sum(),
    size_sqr_sum_own=(x['size'] ** 2).sum(),
    group_n = len(x)
)).reset_index(drop=True)

# グループ化を解除
data = data.reset_index(drop=True)

# マーケットレベル（year）での、各製品属性の和を計算。
data = data.groupby('year').apply(lambda x: x.assign(
    hppw_sum_mkt=x['hppw'].sum(),
    FuelEfficiency_sum_mkt=x['FuelEfficiency'].sum(),
    size_sum_mkt=x['size'].sum(),
    hppw_sqr_sum_mkt=(x['hppw'] ** 2).sum(),
    FuelEfficiency_sqr_sum_mkt=(x['FuelEfficiency'] ** 2).sum(),
    size_sqr_sum_mkt=(x['size'] ** 2).sum(),
    mkt_n=len(x)
)).reset_index(drop=True)

# グループ化を解除
data = data.reset_index(drop=True)

# BLP操作変数の構築

data = data.assign(iv_BLP_own_hppw = lambda x: x['hppw_sum_own']-x['hppw'])
data = data.assign(iv_BLP_own_FuelEfficiency = lambda x: x['FuelEfficiency_sum_own']-x['FuelEfficiency'])
data = data.assign(iv_BLP_own_size = lambda x: x['size_sum_own']-x['size'])
data = data.assign(iv_BLP_other_hppw = lambda x: x['hppw_sum_mkt']-x['hppw_sum_own'])
data = data.assign(iv_BLP_other_FuelEfficiency = lambda x: x['FuelEfficiency_sum_mkt']-x['FuelEfficiency_sum_own'])
data = data.assign(iv_BLP_other_size = lambda x: x['size_sum_mkt']-x['size_sum_own'])

# Differentiation IVの構築
data = data.assign(iv_GH_own_hppw = lambda x: (x['group_n']-1) * x['hppw']**2 + 
                   (x['hppw_sqr_sum_own']-x['hppw']**2) 
                   - 2*x['hppw']*(x['hppw_sum_own']-x['hppw']))

data = data.assign(iv_GH_own_FuelEfficiency = lambda x: (x['group_n']-1) * x['FuelEfficiency']**2 + 
                   (x['FuelEfficiency_sqr_sum_own']-x['FuelEfficiency']**2) 
                   - 2*x['FuelEfficiency']*(x['FuelEfficiency_sum_own']-x['FuelEfficiency']))

data = data.assign(iv_GH_own_size = lambda x: (x['group_n']-1) * x['size']**2 + 
                   (x['size_sqr_sum_own']-x['size']**2) 
                   - 2*x['size']*(x['size_sum_own']-x['size']))

data = data.assign(iv_GH_other_hppw = lambda x: (x['mkt_n']-x['group_n']) * x['hppw']**2 + 
                   (x['hppw_sqr_sum_mkt']-x['hppw_sqr_sum_own']) 
                   - 2*x['hppw']*(x['hppw_sum_mkt']-x['hppw_sum_own']))

data = data.assign(iv_GH_other_FuelEfficiency = lambda x: (x['mkt_n']-x['group_n']) * x['FuelEfficiency']**2 + 
                   (x['FuelEfficiency_sqr_sum_mkt']-x['FuelEfficiency_sqr_sum_own']) 
                   - 2*x['FuelEfficiency']*(x['FuelEfficiency_sum_mkt']-x['FuelEfficiency_sum_own']))

data = data.assign(iv_GH_other_size = lambda x: (x['mkt_n']-x['group_n']) * x['size']**2 + 
                   (x['size_sqr_sum_mkt']-x['size_sqr_sum_own']) 
                   - 2*x['size']*(x['size_sum_mkt']-x['size_sum_own']))

# 不要な列を削除する
columns_to_remove = [col for col in data.columns if (
    col.startswith("sum_own") or
    col.startswith("sum_mkt") or
    col.startswith("sqr_sum_own") or
    col.startswith("sqr_sum_mkt") or
    col == "mkt_n" or
    col == "group_n"
)]
data = data.drop(columns=columns_to_remove)

# Logit Modelにおける価格弾力性
## 日評自動車のデータセットを用意

In [4]:
# 車種のID
IDvec = sorted(data['NameID'].unique())
J = len(IDvec)

# 乱数のseedを固定
# PythonとRでは乱数のseedを同じにしても同じ結果はでてこない
np.random.seed(125)
# IDvecから30個のサンプルを非復元抽出して取得
NIPPYOautoIDvec = np.random.choice(IDvec, size=30, replace=False)

# サンプルを昇順にソート
NIPPYOautoIDvec.sort()

# NIPPYOautoIDvecに含まれるIDを持つ行を抽出
data_NIPPYO = data[data['NameID'].isin(NIPPYOautoIDvec)]



# 必要な列を選択
data_NIPPYO = data_NIPPYO[['Sales', 'price', 'hppw', 'FuelEfficiency', 'size', 'year', 'NameID', 'share']]
# 列 'Sales' と 'price' の対数を計算し、新しい列に追加
data_NIPPYO['log_sales'] = np.log(data_NIPPYO['Sales'])
data_NIPPYO['log_price'] = np.log(data_NIPPYO['price'])

## Logit Modelの価格弾力性行列を作成する
- Logit Modelにおける需要の価格弾力性 $\eta_{jlt}$ は、以下の通り。
$$ \begin{align}
\eta_{jlt} = \frac{\partial q_{jt}}{\partial p_{lt}}\frac{p_{lt}}{q_{jt}} = \left\{
\begin{array}{ll}
-\alpha p_{jt}(1-s_{jt}) & l = j \\
\alpha p_{lt}s_{lt} & l \ne j
\end{array}
\right.
\end{align} $$ 
- 財 $l$ の価格が上昇した時の他の財の需要の変化率は、すべての 財　$j \ne l$ で等しくなる。
- Logit Modelで得られるマーケットシェアは、選択確率と等しくなるため以下の通り。

$$
\begin{align}
s_{jt}  =\left\{
\begin{array}{ll}
\frac{\exp(\delta_{jt})}{1+\sum_{l=1}^{J_t}\exp(\delta_{jt})} & j=1,\cdots, J_t\\
\frac{1}{1+\sum_{l=1}^{J_t}\exp(\delta_{jt})} & j=0
\end{array}
\right.
\end{align}
$$

- よって、任意の2財、$j$と$l$のマーケットシェアの比率は以下の通り。
$$
\frac{s_j}{s_l} = \frac{\exp(\delta_j)}{\exp(\delta_l)}
$$

- これは、ある財$j$と財$l$を選ぶ比率は、他の選択可能な製品や製品属性とは無関係になることを意味している。
- これを無関係な選択肢からの独立性（independence of irrelevant alternatives: **IIA**）が成立しているという。
- 以下では  IIAが成立していることを確かめる。

In [6]:
data['logit_share'] = np.log(data['share']) - np.log(data['share0'])
# Differentiation IVを用いた結果
formula = 'logit_share ~1+ hppw + FuelEfficiency + size + [price ~ iv_GH_own_hppw + iv_GH_own_FuelEfficiency + iv_GH_own_size +iv_GH_other_hppw + iv_GH_other_FuelEfficiency + iv_GH_other_size]'
mod_1 = IV2SLS.from_formula(formula, data=data)
iv_GH = mod_1.fit(cov_type='unadjusted')

In [24]:
dt2016 = data_NIPPYO[data_NIPPYO['year'] == 2016].sort_values('NameID')
price = dt2016['price']
share = dt2016['share']
NameID = dt2016['NameID']

# 自己弾力性
own_elas = iv_GH.params['price']*price*(1-share)
# 交差弾力性
cross_elas = (-1)*iv_GH.params['price']*price*share
# 価格弾力性行列を作成する
# J = len(own_elas)
# elas_mat = np.tile(cross_elas, (J, 1))


# ランダム係数ロジットモデルの定式化
効用を以下のように考える。
$$ u_{ijt} = \left\{
\begin{array}{ll}
-\alpha_i p_{jt} + \beta_i' x_{jt} + \xi_{jt} + \epsilon_{ijt} & j = 1, \cdots, J_t \\
\epsilon_{i0t} & j = 0
\end{array}
\right. $$

ここで、
- $x_{jt} = (1, x_{jt}^1, \cdots, x_{jt}^K)'$：定数項とK個の製品属性を含む$(K+1)\times 1$のベクトル。
- $\beta_i = (\beta_i^0, \cdots, \beta_i^K)$：$x_{jt}$に対応する$(K+1)\times 1$係数ベクトル。

ランダム係数ロジットモデルでは、価格や製品属性に対する選好$(\alpha_i, \beta_i)$が個人ごとに異なることを許容している。また、個人の選好は以下の二つの要素に分解できる。

- 個人に関して観察される要因を社会経済属性（所得、年齢、世帯サイズなど）
- 観察されないような要因である、選考の異質性

これを定式化すると以下の通り。

$$
\begin{pmatrix}\alpha_i \\ \beta_i \end{pmatrix} = \begin{pmatrix}\alpha \\ \beta \end{pmatrix} + \Pi D_i + \sum v_i, \quad D_i\sim F_t(D),\quad v_i\sim G(v)
$$

ここで、
- $\begin{pmatrix}\alpha \\ \beta \end{pmatrix}$：個人ごとに共通な価格、製品属性への選好を捉えている。
- $D_i$：消費者$i$の社会経済属性の$d\times 1$ベクトル。
- $\Pi$：選好パラメータが各消費者の社会経済属性にどのように依存するかを捉える、$(K+2)\times d$行列。
- $v_i$：観察されない選好の異質性を捉える、$(K+2)\times 1$ベクトル。
- $\sum$：$v_i$の分散共分散行列。$(K+2)\times(K+2)$行列。

- 消費者に関するミクロデータが利用可能であれば、$D_i$の分布を考える必要はないが、市場レベルのデータのみの場合、社会経済属性の分布$F_t(D)$のみが利用可能である。
- 観察されない選好の異質性の分布$G(v)$はその定義から観察されることはないので、分析者が何らかの分布を仮定する。


## 個人の選択確率及びマーケットシェアの導出

### 個人の選択確率
ランダム係数ロジットモデルを以下のように書き換える。

$$
u_{ijt} = \underbrace{\delta_{jt}(\theta_1)}_{\text{消費者間で共通の平均効用}} + \underbrace{\mu_{ijt}(\theta_2)}_{\text{消費者の選好の異質性に依存する効用部分}} + \epsilon_{ijt}
$$

ここで、
- $\theta_1 = (\alpha, \beta),\quad \theta_2 = (\Pi, \sum)$。$\theta_1$を線形パラメター、$\theta_2$を非線形パラメターという。
- $\delta_{jt}(\theta_1) = -\alpha p_{jt} + \beta'x_{jt} + \xi_{jt}$：消費者間で共通の平均効用。
- $\mu_{ijt}(\theta_2) = [\Pi D_i + \sum v_i]'\begin{bmatrix}p_{jt} \\ x_{jt}\end{bmatrix}$：消費者の選好の異質性に依存する効用部分。$\begin{bmatrix}p_{jt} \\ x_{jt}\end{bmatrix}$が$(K+2)\times 1$で、$\Pi D_i + \sum v_i$も$(K+2)\times 1$なので、$\mu_{ijt}(\theta_2)$はスカラー。

例えば、自動車需要推定におけるランダム係数ロジットモデルとして以下のような効用のモデルを考える。（社会経済属性を用いないモデルの例）

$$
\tag{1}
u_{ijt} = \beta_i^0 + \beta_i^{size}size_{jt} + \beta^{fuelefficiency}fuelefficiency_{jt} + \beta^{hppw}hppw_{jt}-\alpha_i p_{jt} + \xi_{jt} + \epsilon_{ijt}
$$

- サイズ、価格、定数項について各個人ごとに異なるランダム係数としている。
- 定数項の解釈として、「何かしらの自動車を購入すること」からの効用を捉えているため、「outside goods」からの効用を捉えている。
- 3つのランダム係数について、それぞれ互いに独立な正規分布に従うと仮定する。すなわち、
$$ 
\beta_i^0 \sim N(\beta^0, (\sigma^0)^2), \quad \beta_i^{size} \sim N(\beta^{size}, (\sigma^{size})^2), \quad \alpha_i^0 \sim N(\alpha, (\sigma^{\alpha})^2)
$$

この時、線形パラメターと非線形パラメターはそれぞれ以下の通り。
- $\theta_1 = (\beta^0, \beta^{size}, \beta^{fuelefficiency}, \beta^{hppw}, \alpha)$
- $\theta_2 = (\sigma^0, \sigma^{size}, \sigma^{\alpha})$：i.e., ランダム係数が従う分布の標準偏差パラメター。


「個人属性」を$(D_i,v_i)$とする。個人属性で条件づけ、outside goodsからの効用を0に基準化すると、個人の選択確率は以下の通り。

$$
s_{ijt}(D_i,v_i) = \frac{\exp(\delta_{jt}(\theta_1) + \mu_{ijt}(\theta_2))}{1+\sum_{j=1}^{J_t}\exp(\delta_{jt}(\theta_1) + \mu_{ijt}(\theta_2))}
$$

この個人の選択確率$s_{ijt}(D_i,v_i)$をすべての個人について足し合わせればマーケットシェアになる。この足し上げは個人属性$(D_i,v_i)$の分布に関する積分より行う。

ある製品$j$のマーケット$t$におけるマーケットシェア$s_{jt}$は以下の通り。

$$
s_{jt} = \int\int s_{ijt}(D_i,v_i)dF_t(D_i)dG(v_i) \equiv S_{jt}\left(\{\delta_{jt}\}_{j=1}^{J_t}; \theta_2\right)
$$

In [5]:
# Markdownでの(1)式を推定することを考える。

,Sales,price,hppw,FuelEfficiency,size,year,NameID,share,log_sales,log_price
7,56487,0.756,0.081690,21.0,7.361209,2006,105,0.001105,10.941766,-0.279714
18,28102,0.900,0.063415,19.0,8.112353,2006,136,0.000550,10.243596,-0.105361
19,71897,1.099,0.059091,19.0,8.187467,2006,168,0.001407,11.182990,0.094401
39,9143,5.220,0.192073,10.0,12.526605,2006,36,0.000179,9.120744,1.652497
49,28419,2.380,0.094767,12.2,15.149835,2006,46,0.000556,10.254813,0.867100
...,...,...,...,...,...,...,...,...,...,...
1790,4900,3.208,0.103896,23.4,13.011337,2016,114,0.000086,8.496990,1.165648
1798,248258,2.429,0.074809,40.8,11.745888,2016,216,0.004359,12.422224,0.887480
1813,17026,1.778,0.093966,22.2,11.443877,2016,227,0.000299,9.742497,0.575489
1820,10903,1.999,0.079646,22.2,9.828100,2016,228,0.000191,9.296793,0.692647
